# Individual Assignment

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf

from scipy.optimize import minimize

plt.style.use("ggplot")

RFR = 0.0441
TRADING_DAYS = 252

In [10]:
tickers = ["AAPL", "MSFT", "AMZN", "META", "TSLA", "NVDA", "MSTR", "JPM", "HD", "JNJ"]
groups = []

for ticker in tickers:
    data = yf.download(ticker, start="2020-01-01", end="2024-12-31", auto_adjust=False)
    data.columns = data.columns.droplevel(1)
    data.insert(0, "Ticker", ticker)
    groups.append(data)

df = pd.concat(groups)
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Price,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2020-01-02,AAPL,72.620842,75.087502,75.150002,73.797501,74.059998,135480400
2020-01-03,AAPL,71.914818,74.357498,75.144997,74.125000,74.287498,146322800
2020-01-06,AAPL,72.487854,74.949997,74.989998,73.187500,73.447502,118387200
2020-01-07,AAPL,72.146942,74.597504,75.224998,74.370003,74.959999,108872000
2020-01-08,AAPL,73.307510,75.797501,76.110001,74.290001,74.290001,132079200
...,...,...,...,...,...,...,...
2024-12-23,JNJ,142.891403,145.270004,145.330002,143.449997,144.630005,8720800
2024-12-24,JNJ,143.461914,145.850006,145.850006,144.330002,145.000000,3164100
2024-12-26,JNJ,143.196320,145.580002,146.369995,145.089996,145.509995,4656300


In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
df.isnull().sum().sum()

np.int64(0)